In [1]:
import tensorflow as tf
from Mobilenet_v1_RCNN import model as modellib
from Mobilenet_v1_RCNN.coco import CocoConfig
import numpy as np
import time

Using TensorFlow backend.


In [2]:
%%capture
this_model = modellib.MaskRCNN("inference", CocoConfig(), 'logs/')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


In [3]:
this_model.keras_model.summary()

Model: "mask_rcnn"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 3 864         input_image[0][0]                
__________________________________________________________________________________________________
conv1_bn (BatchNorm)            (None, None, None, 3 128         conv1[0][0]                      
__________________________________________________________________________________________________
conv1_relu (relu6)              (None, None, None, 3 0           conv1_bn[0][0]                   
__________________________________________________________________________________________

In [4]:
params = {}
def time_params(params: dict, warmups=2, trials=5):
    # generate config
    config = CocoConfig()
    for k,v in params.items():
        assert hasattr(config, k)
        setattr(config, k, v)
    # make model, inputs
    this_model = modellib.MaskRCNN("inference", config, 'logs/')
    input_img = np.random.random((1,224,224,3))
    img_metas = this_model.mold_inputs(np.ones((1,224,224,3)))[1]
    anchors = this_model.get_anchors((224,224,3))[None,:,:]
    # warmup
    for _ in range(2):
        #this_model.detect(input_img)
        this_model.keras_model.predict([input_img, img_metas, anchors])
    # evaluate
    start_time = time.time()
    for _ in range(trials):
        #this_model.detect(input_img)
        this_model.keras_model.predict([input_img, img_metas, anchors])
    elapsed_time = (time.time() - start_time) / trials
    return elapsed_time

In [5]:
time_params(dict(
    # tune for speed
    BACKBONE_STRIDES = [4, 8, 16, 32, 64],
    
    # resize everything to 224x224 by cropping
    IMAGE_RESIZE_MODE = "crop", IMAGE_MIN_DIM = 224, IMAGE_MAX_DIM = 224,
    
    # to encourage high FP, low FN
    ROI_POSITIVE_RATIO = 0.5, # high positive rate encourages low false negative rate
    DETECTION_MAX_INSTANCES = 50, # predict many bboxes
    DETECTION_MIN_CONFIDENCE = 0.5, # decrease min confidence -> more bboxes
))

0.07020397186279297

In [6]:
import keras
keras_model = modellib.MaskRCNN("training", CocoConfig(), 'logs/').keras_model

#with keras.utils.CustomObjectScope(modellib.__dict__):
#    keras.models.clone_model(keras_model)

In [7]:
input_img = tf.placeholder(tf.float32, shape=(1,224,224,3))
img_metas = this_model.mold_inputs(np.ones((1,224,224,3)))[1]
img_metas = tf.placeholder(tf.float32, shape=img_metas.shape)
anchors = this_model.get_anchors((224,224,3))
anchors = tf.placeholder(tf.float32, shape=[1,*anchors.shape])
inputs = [input_img, img_metas, anchors]

keras_model(inputs)

ValueError: Layer mask_rcnn expects 7 inputs, but it received 3 input tensors. Input received: [<tf.Tensor 'Placeholder:0' shape=(1, 224, 224, 3) dtype=float32>, <tf.Tensor 'Placeholder_1:0' shape=(1, 93) dtype=float32>, <tf.Tensor 'Placeholder_2:0' shape=(1, 12543, 4) dtype=float32>]

In [ ]:
tf.reset_default_graph()
with tf.Session() as sess:
    tf.keras.backend.set_session(sess)
    this_model = modellib.MaskRCNN("inference", CocoConfig(), 'logs/')
    
    input_img = tf.placeholder(tf.float32, shape=(1,224,224,3))
    img_metas = this_model.mold_inputs(np.ones((1,224,224,3)))[1]
    img_metas = tf.placeholder(tf.float32, shape=img_metas.shape)
    anchors = this_model.get_anchors((224,224,3))
    anchors = tf.placeholder(tf.float32, shape=[1,*anchors.shape])
    inputs = [input_img, img_metas, anchors]
    #inputs = [tf.constant(t, tf.float32) for t in inputs]

    #sess.run(tf.global_variables_initializer())
    output = this_model.keras_model(inputs)
    #sess.run(output)
    tf.summary.FileWriter(logdir='logdir', graph=sess.graph)

%load_ext tensorboard
%tensorboard --logdir logdir

In [ ]:
inputs = keras.Input((None,784))
model = keras.models.Model([inputs], [keras.layers.Dense(10)(inputs)])

inputs = tf.placeholder(tf.float32, (1,784))
outputs = keras_model([inputs])

In [9]:
keras_model.save_weights('untrained_model.h5')

In [10]:
!pwd

/Users/julian/Documents/GitHub/670/final_proj/rcnn
